# Исследование надежности заемщиков


Своевременное погашение кредита зависит от множества факторов. В рамках данного проекта проанализируем, влияют ли на это семейное положение клиентов, количество детей и уровень дохода. 
    
**Цель исследования** - проверить четыре гипотезы:
    
1. Есть ли зависимость между количеством детей и возвратом кредита в срок
2. Есть ли зависимость между семейным положением и возвратом кредита в срок
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок
4. Как разные цели кредита влияют на его возврат в срок

**Ход исследования**
    
Статистика о платежеспособности клиентов предоставлена банком. О качестве данных ничего не известно, поэтому перед проверкой гипотез понадобится обзор данных и предобработка. 
Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки исправим самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


# Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


В таблице 21 525 строк и 12 столбцов. В таблице представлены данные разных типов: object, int64, float64.

Согласно документации к данным:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

В названиях колонок не выявлены нарушения стиля.

В столбцах, содержащих информацию об общем трудовом стаже заемщиков и их ежемесячном доходе, есть пропущенные значения, их доля составляет 10% от общего числа данных. Количество пропущенных значений в обоих столбцах одинаковое, поэтому можно предположить, что это не ошибка, а реальное отсутствие данных. Клиент банка может не иметь трудового стажа и, соответственно, ежемесячного дохода. Но удалить эти строки мы не можем, иначе потеряем 1/10 данных о семейном положении и количестве детей клиентов.  


Кроме этого мы видим, что в данных о трудовом стаже есть отрицательные значения. Минус явно проставлен ошибочно, поэтому на этапе предобработки данных заменим эти значения на положительные. 
Посмотрим несколько записей из датафрейма, выберем 5 случайных строк. 


In [3]:
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10910,1,NaN,41,среднее,1,Не женат / не замужем,4,M,сотрудник,0,NaN,покупка коммерческой недвижимости
12522,0,386557.606883,57,среднее,1,гражданский брак,1,F,пенсионер,0,198538.540997,операции с жильем
7895,1,-5011.732939,41,среднее,1,женат / замужем,0,M,сотрудник,0,101711.579657,операции с жильем
15729,1,-1728.936706,40,Высшее,0,женат / замужем,0,M,сотрудник,0,172814.828313,сделка с автомобилем
17382,0,340176.954536,62,среднее,1,женат / замужем,0,F,пенсионер,0,125929.346549,на покупку автомобиля



Это позволяет увидеть еще некоторые ошибки в данных. Например, 0 в колонке, где указывается возраст клиента, нарушение стиля при заполнении данных об образовании.
Перейдем ко второму этапу работы - к предобработке данных. 
  

# Шаг 2.1 Заполнение пропусков

Общий трудовой стаж в днях и ежемесячный доход - это количественные переменные. Следовательно, мы можем заполнить недостающие данные средним арифметическим или медианным значением.
Данные по ежемесячному доходу нужны нам для проверки гипотезы о том, влияет ли уровень дохода на возврат кредита в срок. Поэтому, чем точнее будут эти данные, тем лучше. Рассчитаем средний доход в зависимости от рода деятельности и заполним пропуски (значения NaN) соответствующими значениями. 

In [4]:
#заменяем все NaN на 0
data['total_income'] = data['total_income'].fillna(0)
income_type_avg = data.groupby('income_type')['total_income'].transform('mean')

income_type_avg_value = data.groupby('income_type')['total_income'].mean()
data['total_income'].fillna(data.loc[data['income_type']=='пенсионер']['total_income'].median())
data['total_income'].fillna(income_type_avg)

#data.info() #проверка, что все строки в столбце total_income заполнены значениями

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64


Некорректные данные о трудовом стаже мы не можем заменить на медианные значения, т.к. еще на этапе обзора данных обнаружили, что в столбце присутствуют отрицательные значения (аномалии).


# Шаг 2.2 Проверка данных на аномалии и исправления

В таблице обнаружены явно аномальные значения в колонке с данными о трудовом стаже клиентов в днях. Есть как отрицательные значения, так и нереально большие (343 937 дня - это 955 лет). Для начала разберемся, какие значения мы будем считать аномальными:
* Отрицательные - стаж может быть не менее 0;
* Верхнюю границу однозначно нельзя указать, можно только примерно рассчитать, в каких пределах будет находиться нормальное значение. В среднем, трудовой стаж человека составляет 45 лет (с 18 до 63 лет). Есть ситуации, когда люди устраиваются на работу раньше (с 14 или 16 лет, а в отдельных сферах еще раньше), но таких случаев не так много, и в таком возрасте трудоустраивают на какие-то часы, а не полные рабочие дни. Т.е. значение будет невелико, и им можно принебречь. Аналогично и возрастом выхода на пенсию. Для простоты расчетов возьмем пенсионный возраст для мужчин, установленный в РФ - 63 года. 

Таким образом, аномальными значениями будем считать те, которые меньше 0 и больше 16 425 (45 лет*365 = 16 425 дней).

Сначала разберемся с пропущенными значениями возраста клиентов банка.

In [5]:
dob_years_median = data['dob_years'].sort_values().median() 
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_median 
# Теперь разберемся с данными по стажу
data['days_employed'] = data['days_employed'].fillna(0) 
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed']*(-1) 
normal_value = 16425
days_employed_normal = data.loc[data['days_employed'] < normal_value] 
days_employed_median = days_employed_normal['days_employed'].sort_values().median() 

data.loc[data['days_employed'] > normal_value, 'days_employed'] = days_employed_median 

Также выполним проверку по остальным данным таблице на наличие аномалий. В столбцах children, days_employed, dob_years, total_income не должно быть отрицательных значений. 

In [6]:
data['dob_years'].value_counts()
data['children'].value_counts()
def abs_children(children):
    return abs(children)
data['children'] = data['children'].apply(abs_children)
data['children']=data['children'].replace(20,2)

Также проверим значения по полю gender, чтобы убедиться, что там нет непредсказуемых значений:

In [7]:
data['gender'].value_counts()
data=data.drop(data[data['gender']=='XNA'].index) 

По колонке 'family_status' проверим, что данные записаны в одном стиле. Значений должно быть немного, если изначально не было нарушений стиля. Также при выводе части таблицы на экран было заметно, что в  колонке 'education' значения записаны буквами разного размера. Это похоже на человеческий фактор, либо сам клиент, либо сотрудник банка, при заполнении анкеты не обратили внимание на размер шрифта. 

In [8]:
data['family_status'].value_counts()
data['family_status'] = data['family_status'].replace('Не женат / не замужем','не женат / не замужем')
data['education']=data['education'].str.lower()

Мы разобрались с пропущенными данными, избавились от аномальных значений, исправили некоторые проблемы со стилем. 
Но визуально таблица все равно выглядят небрежно. Данные о стаже, общем доходе, возрасте воспринимаются лучше, когда записаны целыми числами. Поэтому следующим шагом заменим тип данных в этих строках. 

# Шаг 2.3. Изменение типов данных

In [9]:
display(data['total_income'].max())

2265604.028722744

In [10]:
data[['dob_years','days_employed']]=data[['dob_years','days_employed']].astype('int8')
data[['total_income']]=data[['total_income']].astype('int32')
data.info() #проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int8  
 2   dob_years         21524 non-null  int8  
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int32 
 11  purpose           21524 non-null  object
dtypes: int32(1), int64(4), int8(2), object(5)
memory usage: 1.8+ MB


Таблица стала выглядеть гораздо лучше, осталось избавиться от дубликатов.

# Шаг 2.4. Удаление дубликатов

Сначала найдем и уберем явные дубликаты.

In [11]:
data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop= True)

Неявные дубликаты были обработаны на предыдущем этапе при исправлении данных, напр., разные стили/буквы в колонках с данными об образовании и семейном положении. Сделав все буквы одинакового размера, мы уже избавились от дубликатов.

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

Затем удалим из исходного датафрейма столбцы education и family_status

In [12]:
education_log = data[['education','education_id']]
family_status_log = data[['family_status','family_status_id']]
education_log = education_log.drop_duplicates().reset_index(drop=True)
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
display(education_log)
display(family_status_log)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [13]:
data=data.drop('education', 1)
data=data.drop('family_status', 1)
data.info() #проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21452 non-null  int64 
 1   days_employed     21452 non-null  int8  
 2   dob_years         21452 non-null  int8  
 3   education_id      21452 non-null  int64 
 4   family_status_id  21452 non-null  int64 
 5   gender            21452 non-null  object
 6   income_type       21452 non-null  object
 7   debt              21452 non-null  int64 
 8   total_income      21452 non-null  int32 
 9   purpose           21452 non-null  object
dtypes: int32(1), int64(4), int8(2), object(3)
memory usage: 1.3+ MB


# Шаг 2.6. Категоризация дохода

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [14]:
def total_income_category(income):
    if 0<=income<30000:
        return 'E'
    if 30001<=income<50000:
        return 'D'
    if 50001<=income<200000:
        return 'C'
    if 200001<=income<1000000:
        return 'B'
    return 'A'
#total_income_category(125000) #проверка
data['total_income_category'] = data['total_income'].apply(total_income_category)


# Шаг 2.7. Категоризация целей кредита

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

In [15]:
def credit_category (purpose):
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образов' in purpose:
        return 'получение образования'
data['purpose_category']=data['purpose'].apply(credit_category)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21452 non-null  int64 
 1   days_employed          21452 non-null  int8  
 2   dob_years              21452 non-null  int8  
 3   education_id           21452 non-null  int64 
 4   family_status_id       21452 non-null  int64 
 5   gender                 21452 non-null  object
 6   income_type            21452 non-null  object
 7   debt                   21452 non-null  int64 
 8   total_income           21452 non-null  int32 
 9   purpose                21452 non-null  object
 10  total_income_category  21452 non-null  object
 11  purpose_category       21452 non-null  object
dtypes: int32(1), int64(4), int8(2), object(5)
memory usage: 1.6+ MB


Таким образом, на этапе предобработки данных было сделано следующее:
* Недостающие и аномальные значения заменены на медианные 
* Были обработаны дубликаты
* Вещественные значения заменены на целые
* Для удобства работы с данными были созданы дополнительные датафреймы
* Доходы разбили на диапазоны и присвоили категории
* Добавили категории для целей кредита

Данные можно использовать для проверки гипотез.

# Ответы на вопросы.

## Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [16]:
children_debt=data.groupby('children').agg({'debt':['count','sum']}) 
display(children_debt)
children_debt_part=children_debt['debt']['sum']/children_debt['debt']['count'] 

debt      
          count   sum
children             
0         14089  1063
1          4855   445
2          2128   202
3           330    27
4            41     4
5             9     0

In [17]:
data_pivot=data.pivot_table(index=['children'],columns='total_income_category',values='debt',aggfunc='sum',fill_value=0)
display(data_pivot) 

total_income_category,A,B,C,D,E
children,,,,,
0,1,220,717,12,113
1,1,88,316,4,36
2,0,43,138,4,17
3,0,4,17,1,5
4,0,1,2,0,1
5,0,0,0,0,0


**Вывод 1:**

Между возвратом кредита в срок и количеством детей определенно есть зависимость. 
В первую очередь, нужно отметить, что клиенты, имеющие 3 и более детей берут значительно меньше кредитов. 
Клиенты, имеющие 5 детей вообще не имеют просрочек по кредитам. Если сравнивать остальных клиентов, то наименьшее количество просрочек у клиентов, не имеющих детей (7,5%).

## Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
family_debt=data.groupby('family_status_id').agg({'debt':['count','sum']})
#display(family_debt)
family_debt_part=family_debt['debt']['sum']/family_debt['debt']['count']
display(family_debt_part)
family_debt_total = family_debt.merge(family_status_log, on='family_status_id',how='left')
display(family_debt_total)

family_status_id
0    0.075452
1    0.093517
2    0.065693
3    0.071130
4    0.097509
dtype: float64

/opt/conda/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,"(debt, count)","(debt, sum)",family_status
0,0,12339,931,женат / замужем
1,1,4149,388,гражданский брак
2,2,959,63,вдовец / вдова
3,3,1195,85,в разводе
4,4,2810,274,не женат / не замужем


In [19]:
data_pivot=data.pivot_table(index=['family_status_id'],columns='total_income_category',values='debt',aggfunc='sum',fill_value=0)
display(data_pivot)

total_income_category,A,B,C,D,E
family_status_id,,,,,
0,1,203,628,12,87
1,1,64,269,5,49
2,0,9,47,0,7
3,0,18,56,2,9
4,0,62,190,2,20


**Вывод 2:**

При анализе данных о семейном статусе и просрочкам по кредитам можно увидеть следующее:
* Наименьшее количество кредитов и просрочек по долгу у клиентов, которые ранее были в отношениях, но затем либо развелись, либо потеряли супруга/супругу
* Доля просрочек у людей, состоящих в официальном браке составляет 7,5% - это не такой плохой показатель, он практически на том же уровне, что и у клиентов в разводе
* Самая высокая доля просрочек (более 9%) у клиентов, находящихся в гражданском браке и не женатых/замужних. Это связано с тем, что финансовые вопросы в неофициальных отношениях решаются сложнее всего. В случае, если люди расходятся, обязательство по выплате кредитных обязательств ложится только на того, на кого оформлен кредит. 

## Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
total_income_debt=data.groupby('total_income_category').agg({'debt':['sum','count']})
total_income_debt_part=total_income_debt['debt']['sum']/total_income_debt['debt']['count']
display(total_income_debt_part)
#Построим сводную таблицу просрочек в зависимости от уровня дохода клиента:
data_pivot=data.pivot_table(index=['income_type'],columns='total_income_category',values='debt',aggfunc='sum',fill_value=0)
display(data_pivot)


total_income_category
A    0.080000
B    0.070635
C    0.085532
D    0.060000
E    0.080979
dtype: float64

total_income_category,A,B,C,D,E
income_type,,,,,
безработный,0,0,1,0,0
в декрете,0,0,1,0,0
госслужащий,0,18,60,1,7
компаньон,2,110,233,1,30
пенсионер,0,31,141,7,37
предприниматель,0,0,0,0,0
сотрудник,0,197,754,12,98
студент,0,0,0,0,0


**Вывод 3:**

При анализе данных по уровню дохода и количеством просрочек по кредитам, можно сделать следующие выводы:
* Наименьшее число просрочек по кредитам (6%) отмечается среди людей с доходом 30-50 тыс.руб. И по роду деятельности это, в основном, наемные сотрудики и пенсионеры. Низкий показатель просрочек можно объяснить тем, что люди, имеющие невысокий, но стабильный доход, умеют распределять финансовую нагрузку и берут кредит действительно на то, что им нужно, реально оценивая свои возможности.  
* Чуть выше показатель (7%) в категории клиентов с доходом 200–1.000 тыс. руб. Люди с таким доходом понимают ценность денег и не хотят портить кредитную историю. 
* Хуже показатели по категориям клиентов с совсем низкими доходами и с самыми высокими. Доля просрочек в обоих случаях 8%. Это можно объяснить тем, что в первом случае люди не могут реально оценить свои возможности и просчитать риски, во втором - люди те так внимательно относятся к платежам по кредиту, зная, что в любой момент могут погасить и задолженность, и проценты. 
* Самый высокий показатель просрочек по кредиту у клиентов с доходами 50-200 тыс. руб. - 8,5%. Чтобы понять причину, можно дополнительно проанализировать остальные данные по этим клиентам: их возраст и семейное положение.

## Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [21]:
purpose_debt=data.groupby('purpose_category').agg({'debt':['sum','count']})
purpose_part=purpose_debt['debt']['sum']/purpose_debt['debt']['count']
purpose_part

purpose_category
операции с автомобилем      0.093590
операции с недвижимостью    0.072340
получение образования       0.092200
проведение свадьбы          0.080069
dtype: float64

**Вывод 4:**

Из полученных данных можно сделать следующие выводы:
* Наименьшее число просрочек по кредитам, выданным на операции с недвижимостью. Это может быть связано с тем, что клиенты лучше оценивают свои возможности и просчитывают все риски, т.к. срок кредита очень большой. Также невысокая доля просрочек по кредитам на проведение свадьбы. 
* Наибольшее число просрочек имееют кредиты на образование и автокредиты. В случаях с авто, это может быть связано с тем, что люди оформляют кредит, не до конца вникая в суть и условия. Такие сделки заключаются, как правило, прямо в дилерском центре и связаны с тем, что при покупке авто в кредит покупатель получает дополнительную скидку. 

# Общий вывод

In [22]:
data_pivot=data.pivot_table(index=['purpose_category','income_type'],columns='total_income_category',values='debt',aggfunc='sum',fill_value=0)
display(data_pivot.head(50))

total_income_category                     A   B    C  D   E
purpose_category         income_type                       
операции с автомобилем   в декрете        0   0    1  0   0
                         госслужащий      0   4   16  0   2
                         компаньон        0  24   53  1   7
                         пенсионер        0   7   32  4   8
                         сотрудник        0  51  169  5  19
операции с недвижимостью безработный      0   0    1  0   0
                         госслужащий      0   9   24  0   3
                         компаньон        1  52  106  0   8
                         пенсионер        0  15   59  1  17
                         предприниматель  0   0    0  0   0
                         сотрудник        0  96  344  5  41
                         студент          0   0    0  0   0
получение образования    госслужащий      0   4   14  1   2
                         компаньон        1  22   41  0   8
                         пенсионер        0   6   35  1   6
                         сотрудник        0  38  168  0  23
проведение свадьбы       госслужащий      0   1    6  0   0
                         компаньон        0  12   33  0   7
                         пенсионер        0   3   15  1   6
                         предприниматель  0   0    0  0   0
                         сотрудник        0  12   73  2  15

По результатам исследования можно сделать следующие выводы:
* Выявлена зависимость между возвратом кредита в срок и количеством детей. Наименьшее количество просрочек у клиентов, не имеющих детей или имеющих более 3х детей.
* Наибольшее количество просрочек выявлено у клиентов:
    * находящихся в гражданском браке и не женатых/замужних. 
    * с самыми высокими и самыми низкими доходами. 
    Требуется дополнительный анализ, чтобы понять причину просрочек у клиентов с доходом 50-200 тыс.руб. - у них самый высокий показатель несвоевременного погашения долга
    * с целью кредита на покупку авто или получение образования. 
    
* Наименьшее количество просрочек выявлено у клиентов:
    * которые ранее были в отношениях, но затем либо развелись, либо потеряли супруга/супругу
    * с доходом 30-50 тыс.руб.
    * с целью кредита, выданным на операции с недвижимостью.
Таким образом, можно сделать вывод, что самыми надежными заемщиками для банка являются клиенты, которые берут займ на покупку недвижимости (ипотеку), зарегистрированы в официальном браке и имеют несколько детей.